# Define Data

In [ ]:
import numpy as np
import pandas as pd 

import h2o
from h2o.automl import H2OAutoML

TRAIN_PATH = "../input/spaceship-titanic/train.csv"
TEST_PATH = "../input/spaceship-titanic/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/spaceship-titanic/sample_submission.csv"
SUBMISSION_PATH = "submission.csv "

ID = "PassengerId"
TARGET = "Transported"

SEED_LIST = [7,77,777,7777,77777]
MAX_RUNTIME_SECS = 60 * 3

# Build Model

In [ ]:
h2o.init()

train = h2o.import_file(TRAIN_PATH)
test = h2o.import_file(TEST_PATH)

x = train.columns
y = TARGET

x.remove(y)
x.remove(ID) #remove id  

pred_test = []
for selSeed in SEED_LIST:
    aml_y = H2OAutoML(max_runtime_secs=MAX_RUNTIME_SECS, seed=selSeed)
    aml_y.train(x=x, y=y, training_frame=train)

    preds_y = aml_y.predict(test)
    pred_test.append(preds_y.as_data_frame().predict) 

# Predict Data

In [ ]:
from scipy import stats
modeResult = stats.mode(pred_test, axis=0)
final_test_pred = modeResult.mode

submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] = (final_test_pred[0] > 0.5).astype(bool)
submission.to_csv(SUBMISSION_PATH, index=False)
submission.head()